<a href="https://colab.research.google.com/github/kmk4444/Knowledge-graph-for-rag/blob/main/4_construct_kg_from_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 4: Constructing a Knowledge Graph from Text Documents

### Import packages and set up Neo4j

In [ ]:
!pip install langchain_community neo4j python-dotenv langchain-openai

In [3]:
from dotenv import load_dotenv
import os

# Common data processing
import json
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI


# Warning control
import warnings
warnings.filterwarnings("ignore")

### Take a look at a Form 10-K json file



In [8]:
first_file_name = "/content/drive/MyDrive/0000950170-23-027948.json"

In [9]:
first_file_as_object = json.load(open(first_file_name))

In [ ]:
type(first_file_as_object)

In [ ]:
for k,v in first_file_as_object.items():
    print(k, type(v))

In [12]:
item1_text = first_file_as_object['item1']

In [ ]:
item1_text[0:]